In [14]:
import requests
from datetime import datetime
import time
import csv
from io import StringIO
import logging
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

BASE_URL = "https://www.ndbc.noaa.gov/data/realtime2/"

# Updated Stations relevant to Panama
STATIONS = {
    "42058": "Western Caribbean Sea",
    "32488": "Southwest Caribbean Sea",
    "43502": "East Pacific Ocean",
    "32411": "Eastern Pacific Ocean",
    "42039": "Caribbean Sea"
}


# SQLite database file
DB_FILE = "panama_surf.db"

def fetch_ndbc_data(station_id, data_type):
    url = f"{BASE_URL}{station_id}.{data_type}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.RequestException as e:
        logging.error(f"Failed to fetch data for station {station_id}, type {data_type}: {e}")
        return None

def parse_data(raw_data, data_type):
    parsed_data = []
    csv_data = csv.reader(StringIO(raw_data), delimiter=' ', skipinitialspace=True)
    next(csv_data)  # Skip header
    next(csv_data)  # Skip units
    for row in csv_data:
        if data_type == "txt":
            parsed_data.append({
                "date": f"{row[0]}-{row[1]}-{row[2]}",
                "time": f"{row[3]}:00",
                "wind_dir": row[5],
                "wind_speed": row[6],
                "wave_height": row[8],
                "dominant_wave_period": row[9],
                "average_wave_period": row[10],
                "wave_direction": row[11],
                "sea_level_pressure": row[12],
                "air_temp": row[13],
                "sea_surface_temp": row[14],
            })
        elif data_type == "spec":
            parsed_data.append({
                "date": f"{row[0]}-{row[1]}-{row[2]}",
                "time": f"{row[3]}:00",
                "significant_wave_height": row[5],
                "swell_height": row[6],
                "swell_period": row[7],
                "wind_wave_height": row[8],
                "wind_wave_period": row[9],
                "swell_direction": row[10],
                "wind_wave_direction": row[11],
                "steepness": row[12],
                "average_wave_period": row[13],
            })
    return parsed_data

def create_table(conn, station_id, data_type):
    cur = conn.cursor()
    table_name = f"{station_id}_{data_type}"
    
    if data_type == "txt":
        cur.execute(f"""
            CREATE TABLE IF NOT EXISTS "{table_name}" (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                time TEXT,
                wind_dir TEXT,
                wind_speed TEXT,
                wave_height TEXT,
                dominant_wave_period TEXT,
                average_wave_period TEXT,
                wave_direction TEXT,
                sea_level_pressure TEXT,
                air_temp TEXT,
                sea_surface_temp TEXT
            )
        """)
    elif data_type == "spec":
        cur.execute(f"""
            CREATE TABLE IF NOT EXISTS "{table_name}" (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                time TEXT,
                significant_wave_height TEXT,
                swell_height TEXT,
                swell_period TEXT,
                wind_wave_height TEXT,
                wind_wave_period TEXT,
                swell_direction TEXT,
                wind_wave_direction TEXT,
                steepness TEXT,
                average_wave_period TEXT
            )
        """)
    conn.commit()

# Also update the store_data function to use quoted table names
def store_data(data, station_id, data_type):
    conn = sqlite3.connect(DB_FILE)
    create_table(conn, station_id, data_type)
    cur = conn.cursor()
    
    table_name = f"{station_id}_{data_type}"
    
    # Insert data
    for row in data:
        placeholders = ', '.join(['?' for _ in row])
        columns = ', '.join(f'"{col}"' for col in row.keys())
        sql = f'INSERT INTO "{table_name}" ({columns}) VALUES ({placeholders})'
        cur.execute(sql, list(row.values()))
    
    conn.commit()
    cur.close()
    conn.close()

# Update the get_data function as well
def get_data(station_id, data_type):
    conn = sqlite3.connect(DB_FILE)
    query = f'SELECT * FROM "{station_id}_{data_type}" ORDER BY date DESC, time DESC LIMIT 100'
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

In [16]:
# Fetch and store data for all stations and data types
for station_id, location in STATIONS.items():
    for data_type in ["txt", "spec"]:
        raw_data = fetch_ndbc_data(station_id, data_type)
        if raw_data:
            parsed_data = parse_data(raw_data, data_type)
            store_data(parsed_data, station_id, data_type)
            print(f"Data fetched and stored for {station_id} ({location}), type {data_type}")
        else:
            print(f"Failed to fetch data for {station_id} ({location}), type {data_type}")

# Function to check if data exists for a given station and data type
def check_data_exists(station_id, data_type):
    conn = sqlite3.connect(DB_FILE)
    cur = conn.cursor()
    cur.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{station_id}_{data_type}'")
    exists = cur.fetchone() is not None
    conn.close()
    return exists

# Example: Get and display recent data for station 42058
station_id = "42058"

# Check if data exists before trying to retrieve it
if check_data_exists(station_id, "txt") and check_data_exists(station_id, "spec"):
    txt_data = get_data(station_id, "txt")
    spec_data = get_data(station_id, "spec")

    print("Recent meteorological data:")
    display(txt_data.head())

    print("\nRecent spectral wave data:")
    display(spec_data.head())

    # Example: Plot wave height over time for station 42058
    plt.figure(figsize=(12, 6))
    plt.plot(pd.to_datetime(txt_data['date'] + ' ' + txt_data['time']), txt_data['wave_height'].astype(float))
    plt.title(f"Wave Height Over Time - Station {station_id}")
    plt.xlabel("Date and Time")
    plt.ylabel("Wave Height (m)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Example: Plot wind speed and direction for station 42058
    plt.figure(figsize=(12, 6))
    plt.scatter(txt_data['wind_dir'].astype(float), txt_data['wind_speed'].astype(float))
    plt.title(f"Wind Speed and Direction - Station {station_id}")
    plt.xlabel("Wind Direction (degrees)")
    plt.ylabel("Wind Speed (m/s)")
    plt.tight_layout()
    plt.show()
else:
    print(f"No data available for station {station_id}. Please ensure data has been fetched and stored.")

Data fetched and stored for 42058 (Western Caribbean Sea), type txt


2024-09-29 21:35:05,139 - ERROR - Failed to fetch data for station 32488, type txt: 404 Client Error: Not Found for url: https://www.ndbc.noaa.gov/data/realtime2/32488.txt


Data fetched and stored for 42058 (Western Caribbean Sea), type spec
Failed to fetch data for 32488 (Southwest Caribbean Sea), type txt


2024-09-29 21:35:05,464 - ERROR - Failed to fetch data for station 32488, type spec: 404 Client Error: Not Found for url: https://www.ndbc.noaa.gov/data/realtime2/32488.spec


Failed to fetch data for 32488 (Southwest Caribbean Sea), type spec


...

,id,date,time,wind_dir,wind_speed,wave_height,dominant_wave_period,average_wave_period,wave_direction,sea_level_pressure,air_temp,sea_surface_temp
0,12871,2024-09-30,04:00,120,4.0,MM,MM,MM,MM,1011.4,28.9,30.1
1,12872,2024-09-30,04:00,120,4.0,MM,MM,MM,MM,1011.5,28.9,30.1
2,12873,2024-09-30,03:00,120,4.0,1.1,6,4.9,87,1011.5,29.0,30.1
3,12874,2024-09-30,03:00,120,4.0,1.1,MM,4.9,87,1011.6,29.0,30.1
4,12875,2024-09-30,03:00,120,3.0,MM,MM,MM,MM,1011.8,29.0,30.1



Recent spectral wave data:


,id,date,time,significant_wave_height,swell_height,swell_period,wind_wave_height,wind_wave_period,swell_direction,wind_wave_direction,steepness,average_wave_period
0,4305,2024-09-30,03:00,1.1,1.0,6.2,0.5,4.0,E,ESE,STEEP,4.9
1,4306,2024-09-30,03:00,1.2,1.1,6.7,0.4,4.0,E,E,STEEP,5.1
2,4307,2024-09-30,02:00,1.2,1.1,6.7,0.5,4.0,E,ESE,STEEP,5.1
3,4308,2024-09-30,02:00,1.3,1.2,6.7,0.5,3.6,E,SE,STEEP,5.1
4,1,2024-09-30,01:00,1.1,1.0,6.2,0.5,3.8,E,SE,STEEP,4.9


ValueError: could not convert string to float: 'MM'

<Figure size 1200x600 with 0 Axes>

In [17]:
# Fetch and store data for all stations and data types
active_stations = []
for station_id, location in STATIONS.items():
    station_active = False
    for data_type in ["txt", "spec"]:
        raw_data = fetch_ndbc_data(station_id, data_type)
        if raw_data:
            parsed_data = parse_data(raw_data, data_type)
            store_data(parsed_data, station_id, data_type)
            print(f"Data fetched and stored for {station_id} ({location}), type {data_type}")
            station_active = True
        else:
            print(f"Failed to fetch data for {station_id} ({location}), type {data_type}")
    if station_active:
        active_stations.append(station_id)

print(f"\nActive stations: {', '.join(active_stations)}")

2024-09-29 21:35:13,155 - ERROR - Failed to fetch data for station 32488, type txt: 404 Client Error: Not Found for url: https://www.ndbc.noaa.gov/data/realtime2/32488.txt
2024-09-29 21:35:13,208 - ERROR - Failed to fetch data for station 32488, type spec: 404 Client Error: Not Found for url: https://www.ndbc.noaa.gov/data/realtime2/32488.spec


Data fetched and stored for 42058 (Western Caribbean Sea), type txt
Data fetched and stored for 42058 (Western Caribbean Sea), type spec
Failed to fetch data for 32488 (Southwest Caribbean Sea), type txt
Failed to fetch data for 32488 (Southwest Caribbean Sea), type spec


2024-09-29 21:35:13,266 - ERROR - Failed to fetch data for station 43502, type txt: 404 Client Error: Not Found for url: https://www.ndbc.noaa.gov/data/realtime2/43502.txt
2024-09-29 21:35:13,323 - ERROR - Failed to fetch data for station 43502, type spec: 404 Client Error: Not Found for url: https://www.ndbc.noaa.gov/data/realtime2/43502.spec
2024-09-29 21:35:13,381 - ERROR - Failed to fetch data for station 32411, type txt: 404 Client Error: Not Found for url: https://www.ndbc.noaa.gov/data/realtime2/32411.txt
2024-09-29 21:35:13,467 - ERROR - Failed to fetch data for station 32411, type spec: 404 Client Error: Not Found for url: https://www.ndbc.noaa.gov/data/realtime2/32411.spec


Failed to fetch data for 43502 (East Pacific Ocean), type txt
Failed to fetch data for 43502 (East Pacific Ocean), type spec
Failed to fetch data for 32411 (Eastern Pacific Ocean), type txt


2024-09-29 21:35:13,535 - ERROR - Failed to fetch data for station 42039, type txt: 404 Client Error: Not Found for url: https://www.ndbc.noaa.gov/data/realtime2/42039.txt
2024-09-29 21:35:13,600 - ERROR - Failed to fetch data for station 42039, type spec: 404 Client Error: Not Found for url: https://www.ndbc.noaa.gov/data/realtime2/42039.spec


Failed to fetch data for 32411 (Eastern Pacific Ocean), type spec
Failed to fetch data for 42039 (Caribbean Sea), type txt
Failed to fetch data for 42039 (Caribbean Sea), type spec

Active stations: 42058


In [18]:
# Function to plot data for a given station
def plot_station_data(station_id):
    if check_data_exists(station_id, "txt") and check_data_exists(station_id, "spec"):
        txt_data = get_data(station_id, "txt")
        spec_data = get_data(station_id, "spec")

        print(f"\nRecent meteorological data for station {station_id}:")
        display(txt_data.head())

        print(f"\nRecent spectral wave data for station {station_id}:")
        display(spec_data.head())

        # Plot wave height over time
        plt.figure(figsize=(12, 6))
        plt.plot(pd.to_datetime(txt_data['date'] + ' ' + txt_data['time']), txt_data['wave_height'].astype(float))
        plt.title(f"Wave Height Over Time - Station {station_id}")
        plt.xlabel("Date and Time")
        plt.ylabel("Wave Height (m)")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

        # Plot wind speed and direction
        plt.figure(figsize=(12, 6))
        plt.scatter(txt_data['wind_dir'].astype(float), txt_data['wind_speed'].astype(float))
        plt.title(f"Wind Speed and Direction - Station {station_id}")
        plt.xlabel("Wind Direction (degrees)")
        plt.ylabel("Wind Speed (m/s)")
        plt.tight_layout()
        plt.show()
    else:
        print(f"No data available for station {station_id}.")

# Plot data for all active stations
for station_id in active_stations:
    plot_station_data(station_id)


Recent meteorological data for station 42058:


,id,date,time,wind_dir,wind_speed,wave_height,dominant_wave_period,average_wave_period,wave_direction,sea_level_pressure,air_temp,sea_surface_temp
0,12871,2024-09-30,04:00,120,4.0,MM,MM,MM,MM,1011.4,28.9,30.1
1,12872,2024-09-30,04:00,120,4.0,MM,MM,MM,MM,1011.5,28.9,30.1
2,19321,2024-09-30,04:00,120,4.0,MM,MM,MM,MM,1011.4,28.9,30.1
3,19322,2024-09-30,04:00,120,4.0,MM,MM,MM,MM,1011.5,28.9,30.1
4,12873,2024-09-30,03:00,120,4.0,1.1,6,4.9,87,1011.5,29.0,30.1



Recent spectral wave data for station 42058:


,id,date,time,significant_wave_height,swell_height,swell_period,wind_wave_height,wind_wave_period,swell_direction,wind_wave_direction,steepness,average_wave_period
0,4305,2024-09-30,03:00,1.1,1.0,6.2,0.5,4.0,E,ESE,STEEP,4.9
1,4306,2024-09-30,03:00,1.2,1.1,6.7,0.4,4.0,E,E,STEEP,5.1
2,6461,2024-09-30,03:00,1.1,1.0,6.2,0.5,4.0,E,ESE,STEEP,4.9
3,6462,2024-09-30,03:00,1.2,1.1,6.7,0.4,4.0,E,E,STEEP,5.1
4,4307,2024-09-30,02:00,1.2,1.1,6.7,0.5,4.0,E,ESE,STEEP,5.1


ValueError: could not convert string to float: 'MM'

<Figure size 1200x600 with 0 Axes>